In [1]:
!python tests.py

...
----------------------------------------------------------------------
Ran 3 tests in 20.204s

OK


In [1]:
import numpy as np
from paths_tools import LatticePaths
from pvar_tools import *
import time